In [32]:
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle

In [33]:
pwd = os.getcwd()
print(pwd)

e:\research\HEA_phase_classification\weighted_features\classification\new_alloy\model_training


In [34]:
# Load the saved scaler and encoder
loaded_scaler = joblib.load(pwd + '/scaler/MinMaxScaler.pkl')
loaded_encoder = joblib.load(pwd + '/scaler/LabelEncoder.pkl')

# Load the saved model
with open(pwd + '/saved_models/RandomForestClassifier_optimized.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [35]:
data = pd.read_excel(pwd + "/new_alloy_data/new_alloy_weighted_features_selected_columns.xlsx")
print(data.shape)

(14820, 32)


In [36]:
features = data.iloc[:, 2:].copy()
feature_names = features.columns
feature_names

Index(['weighted_volume_1', 'weighted_density_1', 'weighted_form_energy_pa_1',
       'weighted_volume_2', 'weighted_density_2', 'weighted_form_energy_pa_2',
       'weighted_volume_3', 'weighted_density_3', 'weighted_form_energy_pa_3',
       'weighted_volume_4', 'weighted_density_4', 'weighted_form_energy_pa_4',
       'weighted_volume_5', 'weighted_density_5', 'weighted_form_energy_pa_5',
       'weighted_volume_6', 'weighted_density_6', 'weighted_form_energy_pa_6',
       'weighted_volume_7', 'weighted_density_7', 'weighted_form_energy_pa_7',
       'weighted_volume_8', 'weighted_density_8', 'weighted_form_energy_pa_8',
       'weighted_volume_9', 'weighted_density_9', 'weighted_form_energy_pa_9',
       'weighted_volume_10', 'weighted_density_10',
       'weighted_form_energy_pa_10'],
      dtype='object')

In [37]:
X = pd.DataFrame(loaded_scaler.transform(features), columns=feature_names)

In [38]:
predicted_phases = loaded_model.predict(X)

In [39]:
loaded_encoder.inverse_transform(predicted_phases)

array(['S', 'S', 'S', ..., 'M', 'M', 'M'], dtype=object)

In [43]:
data["predicted_phases"] = loaded_encoder.inverse_transform(predicted_phases)
data.head()

,HEA_formula,Elements,weighted_volume_1,weighted_density_1,weighted_form_energy_pa_1,weighted_volume_2,weighted_density_2,weighted_form_energy_pa_2,weighted_volume_3,weighted_density_3,...,weighted_volume_8,weighted_density_8,weighted_form_energy_pa_8,weighted_volume_9,weighted_density_9,weighted_form_energy_pa_9,weighted_volume_10,weighted_density_10,weighted_form_energy_pa_10,predicted_phases
0,V0.89Zr0.11Cr0.11W0.11Hf,"['V', 'Zr', 'Cr', 'W', 'Hf']",2.438672,0.370533,-0.003858,0.273558,0.033582,-0.000358,1.954511,0.239114,...,17.485931,1.736579,0.006281,1.959665,0.129136,0.000971,0.069392,0.034018,0.000298,S
1,VZr0.12Cr0.12W0.25Hf,"['V', 'Zr', 'Cr', 'W', 'Hf']",4.405630,0.669394,-0.006970,0.539129,0.066184,-0.000706,1.694860,0.207349,...,15.617302,1.551000,0.005610,1.909359,0.125821,0.000946,0.136758,0.067043,0.000586,S
2,VZr0.12Cr0.12W0.38Hf0.88,"['V', 'Zr', 'Cr', 'W', 'Hf']",5.845922,0.888232,-0.009249,0.812933,0.099797,-0.001064,1.479569,0.181010,...,13.633500,1.353983,0.004897,1.894115,0.124817,0.000938,0.206213,0.101091,0.000884,S
3,VZr0.12Cr0.12W0.5Hf0.75,"['V', 'Zr', 'Cr', 'W', 'Hf']",6.608445,1.004090,-0.010455,1.078258,0.132368,-0.001412,1.271145,0.155512,...,11.712976,1.163250,0.004207,1.909359,0.125821,0.000946,0.273517,0.134085,0.001173,S
4,VZr0.12Cr0.12W0.62Hf0.62,"['V', 'Zr', 'Cr', 'W', 'Hf']",6.828837,1.037577,-0.010804,1.347844,0.165463,-0.001764,1.059305,0.129595,...,9.760971,0.969391,0.003506,1.924789,0.126838,0.000953,0.341902,0.167609,0.001466,S


In [44]:
np.unique(data["predicted_phases"], return_counts=True)

(array(['M', 'S'], dtype=object), array([8929, 5891], dtype=int64))

In [45]:
# Selecting the desired columns
subset_df = data[['HEA_formula', 'predicted_phases']]

# Display the subset DataFrame
subset_df.head()

,HEA_formula,predicted_phases
0,V0.89Zr0.11Cr0.11W0.11Hf,S
1,VZr0.12Cr0.12W0.25Hf,S
2,VZr0.12Cr0.12W0.38Hf0.88,S
3,VZr0.12Cr0.12W0.5Hf0.75,S
4,VZr0.12Cr0.12W0.62Hf0.62,S


In [55]:
df = subset_df.copy()

In [57]:
import re

def get_base_element(hea_string):
    # Split based on element and its fraction
    parsed = re.findall(r'([A-Z][a-z]*)(\d*\.\d+|\d*)', hea_string)
    
    # If no fraction is given, assume it to be 1 (e.g., V in VZr...)
    parsed = [(el, float(frac) if frac else 1.0) for el, frac in parsed]
    
    # Get the max fraction
    max_frac = max(parsed, key=lambda x: x[1])[1]
    
    # Find all elements with the max fraction
    max_elements = [el for el, frac in parsed if frac == max_frac]
    
    return max_elements[0]  # Return the first one in case of a tie

df['Base_element'] = df['HEA_formula'].apply(get_base_element)

df.head()

,HEA_formula,predicted_phases,Base_element
0,V0.89Zr0.11Cr0.11W0.11Hf,S,Hf
1,VZr0.12Cr0.12W0.25Hf,S,V
2,VZr0.12Cr0.12W0.38Hf0.88,S,V
3,VZr0.12Cr0.12W0.5Hf0.75,S,V
4,VZr0.12Cr0.12W0.62Hf0.62,S,V


In [58]:
np.unique(df["Base_element"], return_counts=True)

(array(['Al', 'Co', 'Cr', 'Hf', 'Mo', 'Nb', 'Ta', 'Ti', 'V', 'W', 'Zr'],
       dtype=object),
 array([2390,   84,   84,   84, 2390, 2390, 2390, 2390, 2450,   84,   84],
       dtype=int64))

In [60]:
base_elements = ['V', 'Mo', 'Ti', 'Nb', 'Ta', 'Al']

df = df[df["Base_element"].isin(base_elements)]

In [61]:
np.unique(df["Base_element"], return_counts=True)

(array(['Al', 'Mo', 'Nb', 'Ta', 'Ti', 'V'], dtype=object),
 array([2390, 2390, 2390, 2390, 2390, 2450], dtype=int64))

In [62]:
df.to_excel(pwd + "/new_alloys_and_predicted_phases.xlsx", index=False)